# AutoAssist — Multi‑Agent DIY Auto Repair Workflow (Demo)

**Course:** CIS 568 (Spring 2026)  
**Project:** AutoAssist — guided DIY auto repair workflows with human‑in‑the‑loop safety checks.

## What this demo shows
- How a vague symptom becomes a **structured question**
- Web/knowledge **research & grounding**
- **Verification** (fit/safety checks + confidence)
- A **beginner‑friendly step‑by‑step plan** with parts/tools checklist
- **Escalation** / “when NOT to DIY” behavior

## How to run
1. Run cells top‑to‑bottom
2. Enter a sample issue (e.g., “2015 Honda Civic brake squeal”)
3. Review the agent outputs and the final repair plan

> Tip: Keep v1 scope to common, low‑to‑moderate repairs (battery, brakes, lights, filters, spark plugs, OBD‑II codes).


8) **System design**

Provide the system design at a high level. Describe the workflow: inputs → model/agent actions → outputs → human checks/escalation paths. What data will you need, and where will you get it from? How you will evaluate performance, monitor issues, handle failure modes, and use fallbacks? If relevant, briefly call out latency and cost-to-serve assumptions.

9) **Live demo**

Your demo should support your value proposition (not just “look what ChatGPT can do”). It should be tied directly to your primary use case and should make the before/after obvious. Any demo format is fine (for instance, agent/workflow demo, clickable prototype demo, data/model feasibility demo), but it should demonstrate a realistic scenario. Ideally, the demo shows the user journey, the key moment where AI creates the step-change, and a failure mode when the system is uncertain (guardrails, escalation, human-in-the-loop, etc). Do not use confidential customer data in the demo unless you have explicit permission. Keep a video of your demo handy in case you run into technical issues with the live demo.

#Installing and Getting the AI Ready

In [ ]:
# Installing groq
!pip install aisuite openai groq

In [ ]:
from google.colab import userdata
import os
import aisuite
from openai import OpenAI
from groq import Groq

# read the API_KEYs from Colab Secrets and expose it as an env variable
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

# create the clients
openai_client = OpenAI()
groq_client = Groq()
aisuite_client = aisuite.Client()

# select the models you want to use
OPENAI_MODEL= "gpt-4.1-mini"
AISUITE_OPENAI_MODEL = "openai:"+OPENAI_MODEL
GROQ_MODEL = "llama-3.3-70b-versatile"
AISUITE_GROQ_MODEL = "groq:"+GROQ_MODEL

In [ ]:
# Used to clean up the output's display
from IPython.display import display, HTML, Markdown

def show(title, text):
    display(HTML(f"""
     <h4>{title}</h4>
     <div style="white-space: pre-wrap; padding-left: 24px;">{text}</div>"""))

In [ ]:
# This code cell allows OpenAI to search the web so it can return information after it's training date
import json

def openai_web_search(query: str):
  """
    Perform a web search using OpenAI's built-in web_search tool
    and return raw results.
  """
  resp = openai_client.responses.create(
      model=OPENAI_MODEL,
      input=query,
      tools=[{"type": "web_search"}],
  )
  return json.loads(resp.model_dump_json())

In [ ]:
# This code cell is used to create an run_openai_query function

# used for wall-clock timing for request
import time

# run the system/user prompt using OpenAI API, invoke tools as needed;
# return response and query details (usage stats and tools invoked)
def run_openai_query(system_prompt, user_prompt, tools=[]):
  """
    Execute the query, print the prompt, the response, and detailed usage stats.
    Supports tools.
  """

  # start the time before the API call
  start_time = time.time()

  # make chat-completions request thro aisuite, routed to OpenAI-backed model
  completion = aisuite_client.chat.completions.create(
      model=AISUITE_OPENAI_MODEL, # OpenAI-backed model
      messages=[
          {"role": "system", "content": system_prompt}, # system sets global behavior/instructions
           {"role": "user", "content": user_prompt} # user contains the actual task/prompt
      ],
      tools=tools,
      max_turns=3,
  )

  # compute end-to-end wall-clock time for API request
  elapsed_time = time.time() - start_time

  # response
  response = completion.choices[0].message.content.strip()

  # usage stats
  query_details = {}
  query_details["prompt_tokens"]=completion.usage.prompt_tokens
  query_details["completion_tokens"]=completion.usage.completion_tokens
  query_details["total_tokens"]=completion.usage.total_tokens
  if hasattr(completion.usage, 'total_time'):
    query_details["total_time"]=f"{completion.usage.total_time:.2f}s"
  else:
    query_details["total_time"]=f"{elapsed_time:.2f}s"

  # tools invoked
  tools_invoked=[]
  messages = completion.choices[0].intermediate_messages
  for m in messages or []:
    if hasattr(m, "tool_calls") and m.tool_calls:
      for t in m.tool_calls:
        tools_invoked.append(t.function.name)
  query_details["tools_invoked"]=tools_invoked

  return response, query_details

In [ ]:
# This code cell is used to create run_groq_query function

# used for wall-clock timing for request
import time

# run the system/user prompt using OpenAI API; DOES NOT support tools;
# return response and query details (usage stats and tools invoked)
def run_groq_query(system_prompt, user_prompt):
  """
    Execute the query, print the prompt, the response, and detailed usage stats.
    Does not support tools.
  """

  # start the time before the API call
  start_time = time.time()

  # make chat-completions request thro aisuite, routed to Groq-backed model
  completion = aisuite_client.chat.completions.create(
      model=AISUITE_GROQ_MODEL, # Groq-backed model
      messages=[
          {"role": "system", "content": system_prompt}, # system sets global behavior/instructions
           {"role": "user", "content": user_prompt} # user contains the actual task/prompt
      ],
      max_turns=1,
  )

  # compute end-to-end wall-clock time for API request
  elapsed_time = time.time() - start_time

  # response
  response = completion.choices[0].message.content.strip()

  # usage stats
  query_details = {}
  query_details["prompt_tokens"]=completion.usage.prompt_tokens
  query_details["completion_tokens"]=completion.usage.completion_tokens
  query_details["total_tokens"]=completion.usage.total_tokens
  if hasattr(completion.usage, 'total_time'):
    query_details["total_time"]=f"{completion.usage.total_time:.2f}s"
  else:
    query_details["total_time"]=f"{elapsed_time:.2f}s"

  # tools invoked
  tools_invoked=[]
  messages = completion.choices[0].intermediate_messages
  for m in messages or []:
    if hasattr(m, "tool_calls") and m.tool_calls:
      for t in m.tool_calls:
        tools_invoked.append(t.function.name)
  query_details["tools_invoked"]=tools_invoked

  return response, query_details

#Creating Pieces of the AgenticAI Workflow






In [ ]:
#########################################################################
# Creating the Questioner
# Takes in the user's general quetion about car maintence & car model
#########################################################################

def questioner (question: str):

  # questioner system prompt
  questioner_system_prompt = (
    f"""
    You are a questioner agent for answering the user's question about car maintenance.
    Your job is to rephrase the question that the user asks, not to write prose.
    Be clear, concise, and structured.
    """
  )

  # questioner user prompt
  questioner_user_prompt = (
    f"""
    Rephrase question:{question}
    Make sure that the question has the:
      1. the specific question that the user has about the car maintenance
      2. the car model
    Return the question that the user asked only.
    """
  )

  # use Groq (no tools) to create a question
  question_2, query_details = run_groq_query(questioner_system_prompt, questioner_user_prompt)

  # return the question
  return question_2

In [ ]:
#########################################################################
# Creating the Researcher
# Takes in the user's general quetion about car maintence & car model and researches the topic
#########################################################################

def researcher (question_2:str):

  # researcher system prompt
  researcher_system_prompt = (
    f"""
    You are a research for car maintenance.
    Use web search to find accurate, up-to-date information.
    Return factual notes with sources.
    """
  )

  # research user prompt
  researcher_user_prompt = (
    f"""
    Maitenance plan (outline + checklist):{question_2}
    For each outline section:
      1. Provide instruction to fix the question that the user has about their car model
      2. If needed, provide parts that are needed in order to fix the question the user has and make sure it relates to their car model
    Return concise research notes only.
    """
  )

  # use OpenAI (with web_search) to do the research
  research, query_details = run_openai_query(researcher_system_prompt, researcher_user_prompt, tools=[openai_web_search])

  return research

In [ ]:
#########################################################################
# Creating the Verifier
# Looks at the research that the Research did and double checks if it's correct
#########################################################################

def verifier (question_2: str, research: str):

# ****************** INDENTED THE SECTION FROM HERE ******************
  # verifier system prompt
  verifier_system_prompt = (
    f"""
    You are a critical verifier.
    Your job is to evaluate and suggest improvements, not to rewrite the content.
    Be specific and constructive.
    """
  )

  # verifier user prompt
  verifier_user_prompt = (
    f"""
    Original re-written question from user:{question_2}
    Research from researcher:{research}
    Evaluate on: clarity and accuracy (given the research).
    Provide:
      1. A 1-5 score for each category
      2. Specific, actionable improvement suggestions
      3. Check if the repairs mentioned ties back to the model the user specified
    Do not rewrite the research.
    """
  )

  # use Groq (no tools) provide feedback on the feedback
  feedback, query_details = run_groq_query(verifier_system_prompt, verifier_user_prompt)

  return feedback

In [ ]:
#########################################################################
# Creating the Planner
# Takes in the research and verifier to create a plan for the user
#########################################################################

def planner (research: str, feedback: str):

  # planner system prompt
  planner_system_prompt = (
    f"""
    You are a car repair worker telling a client how to repair a car.
    Keep in mind that the client doesn't know much about car repairs.
    Create a list of instructions to repair the car as well as the tools.
    Preserve the original intent and outline unless told otherwise.
    """
  )

  # planner user prompt
  planner_user_prompt = (
    f"""
    Original blog draft:{research}
    Editor feedback:{feedback}
    Revise the research using the feedback.
    Return the full revised instructions and tools for car maintenance.
    """
  )

  # use OpenAI (no tools) creat the instructions
  instructions, query_details = run_openai_query(planner_system_prompt, planner_user_prompt, tools=[])

  return instructions

In [ ]:
#########################################################################
# Creating the Assistant Researcher
# Takes in any question the user has about the instructions
#########################################################################

def assistant (instructions: str, followUp: str):

  # assistant system prompt
  assistant_system_prompt = (
    f"""
    You are a car repair worker.
    Use web search to find accurate, up-to-date information.
    Return factual notes with sources.
    """
  )

  # assistant user prompt
  assistant_user_prompt = (
    f"""
    Original instructions:{instructions}
    User's follow-up question:{followUp}
    For each outline section:
      1. Provide information about the section that the user has a follow-up question about.
      2. If needed, explain how the part is needed for the repair.
    Return concise research notes only.
    """
  )

  # use OpenAI (no tools) create the follow-up answer
  answer, query_details = run_openai_query(assistant_system_prompt, assistant_user_prompt, tools=[openai_web_search])

  return answer

In [ ]:
#########################################################################
# Creating the Explainer Researcher
# Creates the output of the response
#########################################################################

def explainer (answer: str):

  # assistant system prompt
  explainer_system_prompt = (
    f"""
    You are a car repair worker explaining any concept the user doesn't know about the instructions.
    Keep in mind that the user doesn't know much about car repairs.
    Answer the question the user has about the instructions.
    """
  )

  # explainer user prompt
  explainer_user_prompt = (
    f"""
    Follow-up answer:{answer}
    Double check if the feedback is correct.
    Return an easy to understand but informative response to answer the user's question.
    """
  )

  # use OpenAI (no tools) create the follow-up answer
  explanation, query_details = run_openai_query(explainer_system_prompt, explainer_user_prompt, tools=[])

  return explanation

Using Gradio to create the Instructions

In [ ]:
import gradio as gr

# building the UI
with gr.Blocks(title="AgenticAI Car Repair Instructions") as app:

    # Agent 1 : Questioner
    # *****************************************************
    # Textbox to provide the userQuest input
    question = gr.Textbox(label="Quesiton", placeholder="Enter your car repair question as well as the car model.")

    # Button that triggers the questioner when clicked
    submit_userQuest_btn = gr.Button("Submit Question!")

    # Textbox to display the (editable) questioner output
    question_2 = gr.Textbox(label="The question being asked", lines=10)

    # The question is editable by user
    submit_userQuest_btn.click(fn=questioner, inputs=[question], outputs=[question_2])

    # Button that triggers the researcher when clicked
    submit_question_btn = gr.Button("Research the Repair")


    # Agent 2 : Researcher
    # *****************************************************
    # Textbox to display the (editable) researcher output
    research = gr.Textbox(label="Information from Researcher (editable)", lines=10)

    # When the submit question button is clicked, call researcher
    submit_question_btn.click(fn=researcher, inputs=[question_2], outputs=[research])

    # Button that triggers the Verifier when clicked
    submit_research_btn = gr.Button("Double Check the Research")


    # Agent 3 : Verifier
    # *****************************************************
    # Textbox to display the (editable) verifier output
    feedback = gr.Textbox(label="Verified Information (editable)", lines=10)

    # When the submit research button is clicked, call verifier
    submit_research_btn.click(fn=verifier, inputs=[question_2, research], outputs=[feedback])

    # Button that triggers the Planner when clicked
    submit_feedback_btn = gr.Button("Create Repair Instructions")


    # Agent 4 : Planner
    # *****************************************************
    # Textbox to display the (editable) Planner instructions
    instructions = gr.Textbox(label="Instructions (editable)", lines=10)

    # When the submit feedback button is clicked, call Planner
    submit_feedback_btn.click(fn=planner, inputs=[research, feedback], outputs=[instructions])


    # Agent 5 : Assist Researcher
    # *****************************************************
    # Textbox to provide the followUp input
    followUp = gr.Textbox(label="Follow-up Quesiton?", placeholder="Any follow-up questions about the Instructions provided?.")

    # Button that triggers the assistant when clicked
    submit_followUp_btn = gr.Button("Submit Follow-Up Question!")


    # Textbox to display the (editable) assistant output
    answer = gr.Textbox(label="Quick Research on the Follow-Up Question", lines=10)

    # The response is editable by user
    submit_followUp_btn.click(fn=assistant, inputs=[instructions, followUp], outputs=[answer])

    # Button that triggers the researcher when clicked
    submit_answer_btn = gr.Button("Get Explanation")

# Can take Agent 6 out

    # Agent 6 : Explainer
    # *****************************************************
    # Textbox to display Explainer output
    explanation = gr.Textbox(label="Explanation on the Follow-Up Question", lines=10)

    # When the submit feedback button is clicked, call Explainer
    submit_answer_btn.click(fn=explainer, inputs=[answer], outputs=[explanation])

# Start the Gradio web application
app.launch()
